In [1]:

from tensorflow import keras as keras

from drowsiness_detection import config
from drowsiness_detection.data import load_experiment_data
from drowsiness_detection.models import ThreeDStandardScaler
import matplotlib.pyplot as plt
import numpy as np
from contextlib import redirect_stdout
from sklearn.metrics import classification_report
from drowsiness_detection.data import session_type_mapping, label_names_dict

from dl_ts_models.main import create_classifier

In [2]:
# load data
config.set_paths(30, 10)
X_train, X_test, y_train, y_test, cv_splits = load_experiment_data(
    exclude_by="a",
    num_targets=2,
    seed=45,
    test_size=.2,
    split_by_subjects=True,
    use_dummy_data=False,
    nn_experiment=True,
    feature_col_indices=(5, 8, 9, 14, 15, 16, 19),
    model_name="mvts_transformer")

X_train shape: (29012, 300, 7), y_train shape: (29012,)
X_test shape: (7094, 300, 7), y_test shape: (7094,)


In [3]:
num_samples = None
X_train = X_train[:num_samples]
X_test = X_test[:num_samples]
y_test = y_test[:num_samples]
y_train = y_train[:num_samples]



In [4]:
scaler = ThreeDStandardScaler(feature_axis=-1)

X_train_scaled = scaler.fit_transform(X_train, y_train)
X_test_scaled = scaler.transform(X_test)

input_shape = (None, 300, 7)


In [5]:
model = create_classifier(
    classifier_name="resnet",
    input_shape=input_shape[1:],
    nb_classes=1,
    output_directory=str(config.SOURCES_ROOT_PATH.parent.joinpath("data/resnet_experiments")),
    verbose=True
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300, 7)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 300, 64)      3648        ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 300, 64)     256         ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 300, 64)      0           ['batch_normalization[0][0]']

In [8]:

model.fit(x_train=X_train_scaled, y_train=y_train, x_val=X_test_scaled, y_val=y_test, nb_epochs=1, batch_size=256)

114/114 [==============================] - 291s 3s/step - loss: 0.3303 - accuracy: 0.3999 - val_loss: 0.9201 - val_accuracy: 0.4414 - lr: 0.0010


TypeError: predict() missing 1 required positional argument: 'y_test'

In [ ]:
np.argmax([[0], [1]], axis=1)

In [ ]:
from sklearn.metrics import RocCurveDisplay

y_hat = model.predict(X_test_scaled)

RocCurveDisplay.from_predictions( y_pred=y_hat, y_true=y_test, pos_label=1)